In [ ]:
# @title Informação de Memoria do Colab

import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

In [ ]:
# @title Versão do blender (usada pro render) { display-mode: "form" }
Blender_Version = 'Blender 4.4' #@param ["Blender 4.4"]

def path_leaf(path):
    import ntpath
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

dl_link = {
    "Blender 4.4": "https://download.blender.org/release/Blender4.4/blender-4.4.0-linux-x64.tar.xz"
}

dl = dl_link[Blender_Version]
filename = path_leaf(dl)

# Download e instalação do blender
!wget -nc $dl
!mkdir -p ./blender && tar xf $filename -C ./blender --strip-components 1

# Dependencias do COLAB
!apt-get update
!apt-get install -y libboost-all-dev
!apt-get install -y libgl1-mesa-dev
!apt-get install -y libglu1-mesa libsm-dev
!apt-get install -y libxrandr2 libxrandr-dev
!apt-get install -y libxi6 libxi-dev
!apt-get install -y libxxf86vm1 libxxf86vm-dev
!apt-get install -y libxinerama1 libxinerama-dev

# Script de GPU
data = """
import re
import bpy

scene = bpy.context.scene
scene.cycles.device = 'GPU'

prefs = bpy.context.preferences
prefs.addons['cycles'].preferences.get_devices()

cprefs = prefs.addons['cycles'].preferences
print(cprefs)

# Attempt to set GPU device types if available
for compute_device_type in ('CUDA', 'OPTIX', 'OPENCL', 'NONE'):
    try:
        cprefs.compute_device_type = compute_device_type
        print('Device found', compute_device_type)
        break
    except TypeError:
        pass

# Enable all CPU and GPU devices
for device in cprefs.devices:
    if not re.match('intel', device.name, re.I):
        print('Activating', device)
        device.use = True
"""

with open('setgpu.py', 'w') as f:
    f.write(data)



In [ ]:
# @title Adicionar o ACES
!rm "/content/blender/4.4/datafiles/colormanagement/config.ocio"
!wget "https://github.com/AcademySoftwareFoundation/OpenColorIO-Config-ACES/releases/download/v2.0.0/cg-config-v2.0.0_aces-v1.3_ocio-v2.2.ocio" -O "/content/blender/4.4/datafiles/colormanagement/config.ocio"


In [ ]:
# @title Inicia o google drive
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# @title Acessa o google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Teste de render unitario
!sudo ./blender/blender -b '/content/drive/MyDrive/file.blend' -o '/content/drive/My Drive/' -f 1

In [ ]:
# @title  Animação do Frame 1 ao 150
!sudo ./blender/blender -P setgpu.py -b '/content/drive/My Drive/console/renderaqui.blend' -o '/content/drive/My Drive/output_####.png' -s 1 -e 150 -a